In [7]:
import numpy as np
import tensorflow as tf
import time
from tensorflow import keras
import cv2 
# Download the helper library from https://www.twilio.com/docs/python/install
from twilio.rest import Client

project_path = r'F:\projects\security-system'

# Calling
# Setting environment variables
# set TWILIO_ACCOUNT_SID=ACXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# set TWILIO_AUTH_TOKEN=your_auth_token
# account_sid = 'AC0a9374cb2c0f3a81e930d11ff0d4a925'
# auth_token = '22c0ec0ebc3596a42306e73e769000d8'
# client = Client(account_sid, auth_token)

# call = client.calls.create(
#                         url='http://demo.twilio.com/docs/voice.xml',
#                         to='+918087387116',
#                         from_='+14158257003'
#                     )

# print(call.sid)

# Detecting image
batch_size = 32
img_height = 180
img_width = 180

class_names = ['camera_interference', 'object_absent', 'object_present']
export_path = 'saved_models/1617258191'
model = tf.keras.models.load_model(export_path)

image_path = rf'{project_path}\image\image.jpg'


def detect_object(image_path):
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    # print(
    #     "{}: {:.2f} percent confidence."
    #     .format(class_names[np.argmax(score)], 100 * np.max(score))
    # )
    return class_names[np.argmax(score)], 100 * np.max(score)


'''
OpenCV (Open Source Computer Vision Library) is a machine learning library.
The library has more than 2500 optimized algorithms These algorithms can be used to detect and recognize faces, 
identify objects, classify human actions in videos, track camera movements, track moving objects, 
find similar images from an image database, remove red eyes from images taken using flash.
'''

# Camera 0 is the integrated camera port. A port is an endpoint of communication.
camera_port = 1
 
#Number of frames to throw away while the camera adjusts to light levels
ramp_frames = 30
 
# Now we can initialize the camera capture object with the cv2.VideoCapture class.
# All it needs is the index to a camera port.
camera = cv2.VideoCapture(camera_port)
 
    
# Captures a single image from the camera and returns it in PIL format
def get_image():
    retval, im = camera.read()#retval used to return value to cv2. 
    return im


while True:
    # Ramp the camera - these frames will be discarded and are only used to allow v4l2
    # to adjust light levels, if necessary
    for i in range(ramp_frames):
        temp = get_image()

    # Take the actual image we want to keep
    camera_capture = get_image()
    file = rf"{project_path}/image/image.jpg"
    cv2.imwrite(file, camera_capture)
    
    detection = detect_object(image_path)
    
    # Object present
    if detection[0] == 'object_present' and detection[1] > 80:
        print(f'Object is present. Confidence: {detection[1]}')
        
    # Object is stolen(Camera Interference)
    elif detection[0] == 'camera_interference' and detection[1] > 80:
        print(f'Object is stolen(Camera Interference). Confidence: {detection[1]}')
        break
        
    # Object is stolen
    else:
        print(f'Object is stolen. Confidence: {detection[1]}')
        break

        

Object is present. Confidence: 99.99450445175171
Object is present. Confidence: 99.99191761016846
Object is present. Confidence: 99.9969482421875
Object is present. Confidence: 99.99291896820068
Object is present. Confidence: 99.8581051826477
Object is present. Confidence: 99.797922372818
Object is present. Confidence: 99.85536932945251
Object is present. Confidence: 99.78969097137451
Object is stolen. Confidence: 99.9984860420227
